<a href="https://colab.research.google.com/github/raviteja1432/NNDL/blob/main/ASSIGNMENT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import math
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/Share 'Active Power Load - 33_11KV - Godishala Substation  (2).xlsx'")
data.head()

,DATE,TIME,VOLTAGE,CURRENT,PF,POWER (KW),"""WEEKEND/WEEKDAY""",SEASON,Temp (F),Humidity (%),Substation Shutdown,F1,F2,F3,F4,Jul-Oct-,Rainy,0.0
0,2021-01-01,01-00,11.6,102.0,0.96,1967.388015,0.0,1.0,65.0,90.0,NaN,NaN,NaN,NaN,NaN,Nov-Feb,Winter,1.0
1,NaT,02-00,11.6,102.0,0.96,1967.388015,0.0,1.0,65.0,90.0,NaN,NaN,NaN,NaN,NaN,Mar-Jun,Summer,2.0
2,NaT,03-00,11.6,102.0,0.96,1967.388015,0.0,1.0,65.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaT,04-00,11.3,130.0,0.96,2442.607331,0.0,1.0,78.0,49.0,NaN,NaN,NaN,NaN,NaN,Weekend,1.0,NaN
4,NaT,05-00,11.2,148.0,0.96,2756.205522,0.0,1.0,78.0,49.0,NaN,NaN,NaN,NaN,NaN,Weekday,0.0,NaN


In [ ]:
#forming  the dataset 
x = []
y = []
for i in range(25, data.shape[0]):
    x.append(data['POWER (KW)'].iloc[i-25]) #Since x is previous day one hour before of y
    y.append(data['POWER (KW)'].iloc[i])
data = pd.DataFrame({'previous day one hour before Load in kw' : x, 'Present Hour Load in kw' : y})
data.head()

,previous day one hour before Load in kw,Present Hour Load in kw
0,1967.388015,1828.915749
1,1967.388015,1828.915749
2,1967.388015,1892.265507
3,2442.607331,2532.344883
4,2756.205522,3012.157598


In [ ]:
#normalizing
maxX = np.max(data['previous day one hour before Load in kw'])
minX = np.min(data['previous day one hour before Load in kw'])
maxY = np.max(data['Present Hour Load in kw'])
minY = np.min(data['Present Hour Load in kw'])

data['previous day one hour before Load in kw'] = (data['previous day one hour before Load in kw'] - minX) / (maxX - minX)
data['Present Hour Load in kw'] = (data['Present Hour Load in kw']- minY) / (maxY - minY)

In [ ]:
from sklearn.model_selection import train_test_split
x = data['previous day one hour before Load in kw']
y = data['Present Hour Load in kw']
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.1, random_state=101)

In [ ]:
m = np.random.uniform(-3, 3)
c = np.random.uniform(-3, 3)
eta = 0.01
epochs = 500
em2 = 0
ec2 = 0
v = 0.5
E=1E-6

# Loop for epochs
for _ in range(epochs):
    
    for i in range(trainX.shape[0]):
        # Calculating Grad M
        gradM = -1 * (trainY.iloc[i] - m* trainX.iloc[i] - c) * trainX.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (trainY.iloc[i] - m* trainX.iloc[i] - c)
        
        # Calculating updated values of vm and vc
        em2=v*(em2)+(1-v)*(gradM*gradM)
        ec2=v*(ec2)+(1-v)*(gradC*gradC)
        # Updating m and c values
        m -= (eta*gradM)/((em2+E)**0.5)
        c -=  (eta*gradC)/((ec2+E)**0.5)

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.9301248433251077
c = -0.008103115517952687


In [ ]:

train_pred = [m * trainX.iloc[i] + c for i in range(trainX.shape[0])]
test_pred = [m * testX.iloc[i] + c for i in range(testX.shape[0])]

# Denormalization of Output Values
train_pred = [i * (maxY - minY) + minY for i in train_pred]
test_pred = [i * (maxY - minY) + minY for i in test_pred]
trainY = [i * (maxY - minY) + minY for i in trainY]
testY = [i * (maxY - minY) + minY for i in testY]
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
rmse = math.sqrt(mean_squared_error(train_pred, trainY)) 
print("testing error")
print('Root mean square error', rmse) 
mse = (mean_squared_error(train_pred,trainY)) 
print('Mean square error', mse) 
mae=mean_absolute_error(train_pred,trainY)
print('Mean absolute error', mae)


#Testing Accuracies
rmse = math.sqrt(mean_squared_error(test_pred,testY ))
print("testing error") 
print('Root mean square error', rmse) 
mse = (mean_squared_error(test_pred,testY))
print('Mean square error', mse) 
mae=mean_absolute_error(test_pred,testY)
print('Mean absolute error', mae)

testing error
Root mean square error 724.9758200378913
Mean square error 525589.9396396129
Mean absolute error 453.3443135321046
testing error
Root mean square error 685.2637483755387
Mean square error 469586.4048376937
Mean absolute error 431.0055614793888


In [ ]:

#checking the difference between actal value and predicted value of training data
pd.DataFrame({'Actual value' : trainY, 'Predicted value' : train_pred})

,Actual value,Predicted value
0,904.733275,817.889271
1,1699.640673,732.211217
2,1219.952666,920.330422
3,1014.434445,694.545629
4,5011.377243,4207.856417
...,...,...
7856,2180.755890,620.508616
7857,1758.611807,1963.817178
7858,2078.793523,1908.685213
7859,1011.077731,3953.439484


In [ ]:
pd.DataFrame({'Actual' : testY, 'Predicted':test_pred})

,Actual,Predicted
0,1163.615981,905.429891
1,699.194270,1148.767765
2,2281.262641,1959.754856
3,4916.613279,4873.916576
4,1799.593861,794.979704
...,...,...
869,832.652249,941.396690
870,1780.243389,764.247359
871,968.770658,2086.971351
872,1614.714758,1682.306326


In [ ]:

# Real Time Prediction
x = float(input('Enter the power load at one hour and one day before : '))
x = (x - minX) / (maxX - minX)
prediction = m * x + c
prediction = (prediction * (maxY - minY)) + minY
print('Predicted power load at present hour :', prediction)

Enter the power load at one hour and one day before : 9876
Predicted power load at present hour : 9157.419848609787
